In [1]:
import pandas as pd
import numpy as np
import os
import glob
import geopandas as gpd

In [2]:
root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

Import the data into a standard structrue for the geopandas dataframes:

Type | SubType | Name/ID | Easting | Northing

In [3]:
def Import_Points(infile, inputs):
    print(inputs["Type"])
    #Get the before and after columns names
    usecols = list(filter(None,list(inputs.values())[1:8]))
    col_names = list({key: value for key, value in inputs.items() if value in usecols}.keys())

    #Get the data types
    t_dtypes = {key: value for key, value in dtypes.items() if key in col_names}
    dtypes_in = {inputs[k]:v for k, v in t_dtypes.items()}
    dtype_out = {key: value for key, value in dtypes.items() if key in col_names}

    #Import the data
    df = pd.read_csv(infile, usecols=usecols, encoding = "ISO-8859-1", dtype=dtypes_in) 

    #Reformat ready for next step
    df.columns = col_names
    df = df.astype(dtype_out)

    df["Type"] = inputs["Type"]

    return df

In [4]:
#source data
rail_infile = root_path + "NaPTANcsv/RailReferences.csv"
rail_inputs = {"Type":"RailwayStations", "Name":"StationName", "Easting":"Easting", "Northing":"Northing"}

ferry_infile = root_path + "NaPTANcsv/FerryReferences.csv"
ferry_inputs = {"Type":"FerryTerminals", "Name":"Name", "Easting":"Easting", "Northing":"Northing"}

bus_infile = root_path + "NaPTANcsv/Stops.csv"
bus_inputs = {"Type":"BusStops", "Name":"CommonName", "Easting":"Easting", "Northing":"Northing"}

acc2020_infile = root_path + "RoadSafety/dft-road-casualty-statistics-accident-2020.csv"
acc2020_inputs = {"Type":"RoadAccidents", "Name":"accident_reference", "Easting":"location_easting_osgr", "Northing":"location_northing_osgr"}

NSPL_infile = root_path + "NSPL_NOV_2020_UK/Data/NSPL_NOV_2020_UK.csv"
NSPL_inputs = {"Type":"NSPL", "Name":"pcd", "Easting":"oseast1m", "Northing":"osnrth1m"}

#NSUL_Infile defined below
NSUL_inputs = {"Type":"NSUL", "Name":"uprn", "Details_Str":"pcds", "Easting":"gridgb1e", "Northing":"gridgb1n"}

point_pairs = [[rail_infile, rail_inputs],
[ferry_infile, ferry_inputs],
[bus_infile, bus_inputs],
[acc2020_infile, acc2020_inputs],
[NSPL_infile,NSPL_inputs]]

NSUL_Pairs = [[root_path + "NSUL_OCT_2020/Data/" + str(i), NSUL_inputs] for i in os.listdir("D:/GeoData/NSUL_OCT_2020/Data") if ".csv" in i]


point_pairs = point_pairs + NSUL_Pairs


In [5]:
#initiate the standard structure
Points_of_Interest = pd.DataFrame({"Type":"", "Name":"", "Details_Str":"", "Details_Float":0, "Easting":0, "Northing":0}, index=[0])
dtypes = {"Type":"str", "Name":"str", "Details_Str":"str", "Details_Float":np.float64, "Easting":np.float64, "Northing":np.float64}
Points_of_Interest = Points_of_Interest.astype(dtypes)
Points_of_Interest = Points_of_Interest.drop(0)

In [6]:
for l in point_pairs:
    Points_of_Interest = Points_of_Interest.append(Import_Points(l[0], l[1]), ignore_index=True)

#remove the spaces from the postcode
Points_of_Interest.loc[Points_of_Interest["Type"]=="NSPL", "Name"] = Points_of_Interest.loc[Points_of_Interest["Type"]=="NSPL", "Name"].astype(str).str.replace(" ","")


RailwayStations
FerryTerminals
BusStops
RoadAccidents
NSPL


In [ ]:
#Import the land registry data, merge on the postcodes from NSPL data then add to the main file
LReg_Names = ['Transaction_unique_identifier', 'Price', 'Date_of_Transfer', 'Postcode', 'Property_Type', 
              'Old_New', 'Duration', 'PAON', 'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 
              'PPDCategory_Type', 'Record_Status_monthly_file_only']

usecols = ['Transaction_unique_identifier', 'Price', 'Date_of_Transfer', 'Postcode', 'Property_Type', 'Old_New', 'Duration']

LReg_Data2020 = pd.read_csv(root_path + "LandReg/pp-2020.csv", names=LReg_Names, usecols=usecols) 
LReg_Data2021 = pd.read_csv(root_path + "LandReg/pp-2021.csv", names=LReg_Names, usecols=usecols)

LReg_Data = pd.concat([LReg_Data2020, LReg_Data2021])

#remove the spaces from the postcode
LReg_Data["Postcode"] = LReg_Data["Postcode"].astype(str).str.replace(" ","")

LReg_Data = LReg_Data.merge(Points_of_Interest.loc[Points_of_Interest["Type"]=="NSPL", ["Name", "Easting", "Northing"]], left_on="Postcode", right_on="Name")

LReg_Data["Details_Str"] = np.column_stack((LReg_Data["Date_of_Transfer"].to_numpy(),
    LReg_Data["Property_Type"].to_numpy(), 
    LReg_Data["Old_New"].to_numpy(), 
    LReg_Data["Duration"].to_numpy())
    ).tolist()

LReg_Data["Details_Float"] = LReg_Data["Price"]
LReg_Data["Name"] = LReg_Data["Transaction_unique_identifier"]
LReg_Data["Type"] = "LReg"
LReg_Data = LReg_Data.loc[:, ["Type", "Name", "Details_Str", "Details_Float", "Easting", "Northing"]]

Points_of_Interest = Points_of_Interest.append(LReg_Data, ignore_index=True)
del LReg_Data

In [ ]:
#Firestation Data
FireStations = pd.read_excel(root_path + "Fire_data/" + "fire-stations-dataset-121120.ods", 
                         engine="odf",
                        sheet_name = "STATIONS")

FireStations["Type"] = "FireStations"
FireStations["Name"] = FireStations["STATION_NAME"]
FireStations["Easting"] = FireStations["STATION_EASTING"]
FireStations["Northing"] = FireStations["STATION_NORTHING"]
FireStations = FireStations.loc[:, ["Type", "Name", "Easting", "Northing"]]

Points_of_Interest = Points_of_Interest.append(FireStations, ignore_index=True)
del FireStations

In [ ]:
#Schools Data
usecols=["Postcode", "EstablishmentName", "EstablishmentTypeGroup (name)"]
Schools =pd.read_csv(root_path+"Schools/"+"results.csv", encoding = "ISO-8859-1", low_memory=False, usecols=usecols)

#remove the spaces from the postcode
Schools["Postcode"] = Schools["Postcode"].astype(str).str.replace(" ","")

Schools = Schools.merge(Points_of_Interest.loc[Points_of_Interest["Type"]=="NSPL", ["Name", "Easting", "Northing"]], left_on="Postcode", right_on="Name")

Schools["Type"] = "Schools"
Schools["Details_Str"] = Schools["EstablishmentTypeGroup (name)"]
Schools["Name"] = Schools["EstablishmentName"]
Schools = Schools.loc[:, ["Type", "Name", "Details_Str","Easting", "Northing"]]

Points_of_Interest = Points_of_Interest.append(Schools, ignore_index=True)
del Schools

In [ ]:
#Convert to Geopandas dataframe
x_points = Points_of_Interest["Easting"].to_numpy()
y_points = Points_of_Interest["Easting"].to_numpy()

points_gdf = gpd.GeoDataFrame(Points_of_Interest.loc[:,["Type", "Name", "Details_Str", "Details_Float"]], geometry=gpd.points_from_xy(x_points, y_points), crs=Main_CRS)

#deal with NI postcodes

,Type,Name,Details_Str,Details_Float,geometry
0,RailwayStations,Penzance Rail Station,NaN,NaN,POINT (147588.000 147588.000)
1,RailwayStations,St Ives (Cornwall) Rail Station,NaN,NaN,POINT (151947.000 151947.000)
2,RailwayStations,Carbis Bay Rail Station,NaN,NaN,POINT (152930.000 152930.000)
3,RailwayStations,St Erth Rail Station,NaN,NaN,POINT (154150.000 154150.000)
4,RailwayStations,Lelant Saltings Rail Station,NaN,NaN,POINT (154430.000 154430.000)
...,...,...,...,...,...
4578637,Schools,Sporting Chance,Welsh schools,NaN,POINT (332032.000 332032.000)
4578638,Schools,Westward House School,Welsh schools,NaN,POINT (228085.000 228085.000)
4578639,Schools,Ysgol Calon y Dderwen,Welsh schools,NaN,POINT (310669.000 310669.000)
4578640,Schools,Newport Nursery School,Welsh schools,NaN,POINT (331908.000 331908.000)
